## <span style="color: #2F4F4F;">Author: Harpreet Kang</span> 

In [1]:
'''
Capstone Project: CIND820 
Harpreet Kang
July 2, 2024
'''

'\nCapstone Project: CIND820 \nHarpreet Kang\nJuly 2, 2024\n'

## <span style="color: #2F4F4F;">Data Load </span>

 ### <span style="color: #4682B4;">Loading the Libraries</span>



In [84]:
%matplotlib inline
import time
from IPython.display import display
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,precision_score,recall_score,f1_score, accuracy_score, confusion_matrix, classification_report
from scipy import stats
from scipy.stats import friedmanchisquare,kruskal
import statsmodels.api as sm
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# Loading the libraries

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Displaying all the columns and rows

### <span style="color: #4682B4;">Loading the Data</span>


In [4]:
data = pd.read_excel('dataset_lfs_2024.xlsx')
# Loading the data

## <span style="color: #2F4F4F;">Pre-Processing </span>

### <span style="color: #4682B4;">Filtering the Data and Fixing Decimal Places</span>


In [5]:
data.dropna(subset=["HRLYEARN"],inplace=True)
# Removing the null values in HRLYEARN column
data=data[data['MJH']==1]
# Filtering for single job holders
data=data[data['SCHOOLN']==1]
# Filtering for non students
data=data[data['FTPTMAIN']==1]
# Filtering for full-time workers
data=data[data['PERMTEMP']==1]
# Filtering for permanent workers
data['HRLYEARN']=data['HRLYEARN']/100
# The data dictionary for this dataset indicated that the last 2 values of this numeric column were the decimal points. So dividing the HRLYEARN column by 100 will add 2 decimal points. 
data['UHRSMAIN']=data['UHRSMAIN']/10
data['AHRSMAIN']=data['AHRSMAIN']/10
data['UTOTHRS']=data['UTOTHRS']/10
data['ATOTHRS']=data['ATOTHRS']/10
data['HRSAWAY']=data['HRSAWAY']/10
data['PAIDOT']=data['PAIDOT']/10
data['UNPAIDOT']=data['UNPAIDOT']/10
data['XTRAHRS']=data['XTRAHRS']/10
# The data dictionary for this dataset indicated that the last 1 decimial of the 8 numeric columns above were the decimal points
# So dividing the UHRSMAIN column by 00 will add 1 decimal points.
# For HRLYEARN, it was the last two decimal points

In [6]:
#profile1 = ProfileReport(data, title="Profiling Report")
#profile1.to_file("EDA after eliminating blanks of Usual Hourly Wages (HRLYEARN).html")
# Generating the profiling report after removing the null values in HRLYEARN column


#### <span style="color: #4682B4;">Mapping Variables to Discriptions</span> 

In [7]:
full_variable_name_mapping={
'rec_num':'Order of record in file',
'survyear':'Survey year',
'survmnth':'Survey month',
'lfsstat':'Labour force status',
'prov':'Province',
'cma':'Nine largest CMAs',
'age_12':'Five-year age group of respondent',
'age_6':'Age in 2 and 3 year groups, 15 to 29',
'sex':'Sex of respondent',
'marstat':'Marital status of respondent',
'educ':'Highest educational attainment',
'mjh':'Single or multiple jobholder',
'everwork':'Identifies if a person has worked in the last year',
'ftptlast':'Full- or part-time status of last job',
'cowmain':'Class of worker, main job',
'immig':'Immigrant status',
'naics_21':'Industry of main job',
'noc_10':'Occupation at main job (noc_10)',
'noc_43':'Occupation at main job (noc_43)',
'yabsent':'Reason of absence, full week',
'wksaway':'Number of weeks absent from work',
'payaway':'Paid for time off, full-week absence only',
'uhrsmain':'Usual hours worked per week at main job',
'ahrsmain':'Actual hours worked per week at main job',
'ftptmain':'Full- or part-time status at main or only job',
'utothrs':'Usual hours worked per week at all jobs',
'atothrs':'Actual hours worked per week at all jobs',
'hrsaway':'Hours away from work, part-week absence only',
'yaway':'Reason for part-week absence',
'paidot':'Paid overtime hours in reference week',
'unpaidot':'Unpaid overtime hours in reference week',
'xtrahrs':'Number of overtime or extra hours worked',
'whypt':'Reason for part-time work',
'tenure':'Job tenure with current employer',
'prevten':'Job tenure with previous employer',
'hrlyearn':'Usual hourly wages',
'union':'Union status',
'permtemp':'Job permanency',
'estsize':'Establishment size',
'firmsize':'Firm size',
'durunemp':'Duration of unemployment',
'flowunem':'Flows into unemployment',
'unemftpt':'Job seekers by type of work sought and temporary layoffs by work status of last job',
'whylefto':'Reason for leaving job during previous year (whylefto)',
'whyleftn':'Reason for leaving job during previous year (whyleftn)',
'durjless':'Duration of joblessness',
'availabl':'Availability during the reference week',
'lkpubag':'Unemployed, used public employment agency',
'lkemploy':'Unemployed, checked with employers directly',
'lkrels':'Unemployed, checked with friends or relatives',
'lkatads':'Unemployed, looked at job ads',
'lkansads':'Unemployed, placed or answered ads',
'lkothern':'Unemployed, other methods',
'prioract':'Main activity before started looking for work',
'ynolook':'Reason for not looking for work during the reference week',
'tlolook':'Temporary layoff, looked for work during the last four weeks',
'schooln':'Current student status',
'efamtype':'Type of economic family',
'agyownk':'Age of youngest child',
'finalwt':'Standard final weight'}

# Mapping descriptions to the variable names

In [8]:
#data.describe()
# Displaying the summary statistics of the data

In [9]:
#data.info()
# Displaying the data types of the columns

###  <span style="color: #4682B4;">Calculating Missing Values in the Dataset</span>


In [10]:
data_missing_values= data.isnull().sum()
# Counting the missing values in each column
data_percent_missing = round(data.isnull().sum() * 100 / len(data),2)
# Calculating the percentage of missing values in each column
missing_values_percent=pd.DataFrame(data_percent_missing, columns=['Missing Values %'])
# Creating a dataframe to display the missing values percentage
missing_values_percent=missing_values_percent.rename_axis('Variables')
# Renaming the index

In [11]:
#data.head()
# Displaying the first 5 rows of the data

###  <span style="color: #4682B4;">Creating a Function to generate Descriptive Statistics</span>


In [12]:
def statistics_table(data_frame):
    data_type_mapping={
    'rec_num':'Nominal',
    'survyear':'Ordinal',
    'survmnth':'Ordinal',
    'lfsstat':'Nominal',
    'prov':'Nominal',
    'cma':'Nominal',
    'age_12':'Ordinal',
    'age_6':'Ordinal',
    'sex':'Nominal',
    'marstat':'Nominal',
    'educ':'Ordinal',
    'mjh':'Nominal',
    'everwork':'Nominal',
    'ftptlast':'Nominal',
    'cowmain':'Nominal',
    'immig':'Nominal',
    'naics_21':'Nominal',
    'noc_10':'Nominal',
    'noc_43':'Nominal',
    'yabsent':'Nominal',
    'wksaway':'Nominal',
    'payaway':'Nominal',
    'uhrsmain':'Continuous',
    'ahrsmain':'Continuous',
    'ftptmain':'Nominal',
    'utothrs':'Continuous',
    'atothrs':'Continuous',
    'hrsaway':'Continuous',
    'yaway':'Nominal',
    'paidot':'Continuous',
    'unpaidot':'Continuous',
    'xtrahrs':'Continuous',
    'whypt':'Nominal',
    'tenure':'Discrete',
    'prevten':'Discrete',
    'hrlyearn':'Continuous',
    'union':'Nominal',
    'permtemp':'Nominal',
    'estsize':'Ordinal',
    'firmsize':'Ordinal',
    'durunemp':'Discrete',
    'flowunem':'Nominal',
    'unemftpt':'Nominal',
    'whylefto':'Nominal',
    'whyleftn':'Nominal',
    'durjless':'Discrete',
    'availabl':'Nominal',
    'lkpubag':'Nominal',
    'lkemploy':'Nominal',
    'lkrels':'Nominal',
    'lkatads':'Nominal',
    'lkansads':'Nominal',
    'lkothern':'Nominal',
    'prioract':'Nominal',
    'ynolook':'Nominal',
    'tlolook':'Nominal',
    'schooln':'Nominal',
    'efamtype':'Nominal',
    'agyownk':'Ordinal',
    'finalwt':'Continuous'}

    data_object_mapping={
    'rec_num':'Qualitative',
    'survyear':'Qualitative',
    'survmnth':'Qualitative',
    'lfsstat':'Qualitative',
    'prov':'Qualitative',
    'cma':'Qualitative',
    'age_12':'Qualitative',
    'age_6':'Qualitative',
    'sex':'Qualitative',
    'marstat':'Qualitative',
    'educ':'Qualitative',
    'mjh':'Qualitative',
    'everwork':'Qualitative',
    'ftptlast':'Qualitative',
    'cowmain':'Qualitative',
    'immig':'Qualitative',
    'naics_21':'Qualitative',
    'noc_10':'Qualitative',
    'noc_43':'Qualitative',
    'yabsent':'Qualitative',
    'wksaway':'Qualitative',
    'payaway':'Qualitative',
    'uhrsmain':'Quantitative',
    'ahrsmain':'Quantitative',
    'ftptmain':'Qualitative',
    'utothrs':'Quantitative',
    'atothrs':'Quantitative',
    'hrsaway':'Quantitative',
    'yaway':'Qualitative',
    'paidot':'Quantitative',
    'unpaidot':'Quantitative',
    'xtrahrs':'Quantitative',
    'whypt':'Qualitative',
    'tenure':'Quantitative',
    'prevten':'Quantitative',
    'hrlyearn':'Quantitative',
    'union':'Qualitative',
    'permtemp':'Qualitative',
    'estsize':'Qualitative',
    'firmsize':'Qualitative',
    'durunemp':'Qualitative',
    'flowunem':'Qualitative',
    'unemftpt':'Qualitative',
    'whylefto':'Qualitative',
    'whyleftn':'Qualitative',
    'durjless':'Qualitative',
    'availabl':'Qualitative',
    'lkpubag':'Qualitative',
    'lkemploy':'Qualitative',
    'lkrels':'Qualitative',
    'lkatads':'Qualitative',
    'lkansads':'Qualitative',
    'lkothern':'Qualitative',
    'prioract':'Qualitative',
    'ynolook':'Qualitative',
    'tlolook':'Qualitative',
    'schooln':'Qualitative',
    'efamtype':'Qualitative',
    'agyownk':'Qualitative',
    'finalwt':'Quantitative'}
    data_table=data_frame.columns.to_frame(index=False)
    data_table=data_table.rename(columns={0:'Variable Name'})
    columns=list(data_table.iloc[:,0] )
    data_table_mean_lst=[]
    data_table_median_lst=[]
    data_table_min_lst=[]
    data_table_max_lst=[]
    data_table_std_lst=[]
    data_table_object_lst=[]
    data_table_dtype_lst=[]

    for column in columns:
        data_table_mean_lst.append(round(data_frame[column].mean(),2))
        data_table_median_lst.append(round(data_frame[column].median(),2))
        data_table_min_lst.append(round(data_frame[column].min(),2))
        data_table_max_lst.append(round(data_frame[column].max(),2))
        data_table_std_lst.append(round(data_frame[column].std(),2))
        data_table_object_lst.append(data_object_mapping.get((column.lower())))
        data_table_dtype_lst.append(data_type_mapping.get((column.lower())))
    summary_statistics_table=pd.DataFrame(list(zip(columns,data_table_object_lst,data_table_dtype_lst,data_table_mean_lst,data_table_median_lst,data_table_min_lst,data_table_max_lst,data_table_std_lst)),
                                      columns=['Variable','Data','Data Type','Mean','Median','Min','Max','Standard Deviation'])
    return summary_statistics_table

# Created a function to generate the summary statistics table

In [13]:
summary_statistics_table=statistics_table(data)
# Creating the summary statistics table

In [14]:
# Removing mean, and standard deviation from non continous and discrete variables
summary_statistics_table["Mean"] = summary_statistics_table.apply(lambda row: row["Mean"] if row["Data Type"].lower() in ["continuous", "discrete"] else None, axis=1)
summary_statistics_table["Standard Deviation"] = summary_statistics_table.apply(lambda row: row["Standard Deviation"] if row["Data Type"].lower() in ["continuous", "discrete"] else None, axis=1)

# Removing the median from the nominal variables
summary_statistics_table["Median"] = summary_statistics_table.apply(lambda row: None if row["Data Type"].lower() == "nominal" else row["Median"] , axis=1)
summary_statistics_table=summary_statistics_table.merge(missing_values_percent,left_on='Variable',right_index=True)
summary_statistics_table.sort_values(by=['Missing Values %','Variable'],inplace=True)
summary_statistics_table.reset_index(drop=True,inplace=True)

summary_statistics_table_1=summary_statistics_table[['Variable','Data','Data Type']]
summary_statistics_table_1["Description"]=summary_statistics_table_1["Variable"].apply(lambda x: full_variable_name_mapping.get(x.lower()))
summary_statistics_table_1=summary_statistics_table_1[['Variable','Description','Data','Data Type']]
#summary_statistics_table_1.to_excel('Summary Statistics Table_Figure_3.xlsx',index=False)  

summary_statistics_table_2=summary_statistics_table.drop(columns=['Data','Data Type'],axis=1)
#summary_statistics_table_2.to_excel('Summary Statistics Table_Figure_2.xlsx',index=False)  

# Creating a function to generate the statistics table for excel

/var/folders/0f/v6wgxn554tx62wm1nqjqw5hw0000gn/T/ipykernel_2798/565589369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_statistics_table_1["Description"]=summary_statistics_table_1["Variable"].apply(lambda x: full_variable_name_mapping.get(x.lower()))


### <span style="color: #BDB76B;">Feature Selection: Low Variance </span> 


In [15]:
remove_attributes=list(summary_statistics_table[summary_statistics_table_2['Missing Values %']>56.00]['Variable'])
# Creating a list of variables with more than 56% missing values

# Dropping the below variables as they are not important
remove_attributes.append('SURVMNTH')
remove_attributes.append('SURVYEAR')
remove_attributes.append('REC_NUM')
remove_attributes.append('FINALWT')

##################################

remove_attributes.append('LFSSTAT')
# Already filtered for employed
remove_attributes.append('MJH')
# Only looking at single job holders
remove_attributes.append('FTPTMAIN')
# Deleting because we are only looking at full timer workers not parttime
remove_attributes.append('PERMTEMP')
# Already filtered for full time jobs
remove_attributes.append('SCHOOLN')
# Only looking at non students and not full time or part time students

###  <span style="color: #BDB76B;">Feature Selection: Missing Values </span> 


In [16]:
remove_attributes.append('HRSAWAY')
# Hours away from work, part-week absence only low variance has 79 perecent 0s
remove_attributes.append('PAIDOT')
# Paid over time has low variance 83% of zeros
remove_attributes.append('UNPAIDOT')
# Upaid overtime has 83% of zeros
remove_attributes.append('XTRAHRS')
# Contains over 70% of zeroes

In [17]:
data2=data.drop(columns=remove_attributes)
# dropping the variables with more than 56% missing values and creating a new dataframe

###  <span style="color: #4682B4;">Mapping all of the Categorical Variables from the Data Dictionary</span>


In [18]:
data2['SEX']=data2['SEX'].map({1:0,2:1})
# Male is 0 and Female is 1. 
data2['IMMIG']=data2['IMMIG'].map({1:1,2:1,3:0})
# immigrant is 1 and non immigrant is 0data2['MARSTAT']=data2['MARSTAT'].map({1:1,2:0,3:0,4:0,5:0,6:0})
data2['MARSTAT']=data2['MARSTAT'].map({1:1,2:0,3:0,4:0,5:0,6:0})
# Married is 1 and not married is 0
data2['CMA']=data2['CMA'].map({0:0,1:1,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1})
# 1 is for the 9 largest CMAs and 0 is for the rest.

data2['EFAMTYPE']=data2['EFAMTYPE'].map({
    1:'Person not in an economic family',
    2:'Dual-earner couple, no children or none under 25',
    3:'Dual-earner couple, youngest child 0 to 17',
    4:'Dual-earner couple, youngest child 18 to 24',
    5:'Single-earner couple, male employed, no children or none under 25',
    6:'Single-earner couple, male employed, youngest child 0 to 17',
    7:'Single-earner couple, male employed, youngest child 18 to 24',
    8:'Single-earner couple, female employed, no children or none under 25',
    9:'Single-earner couple, female employed, youngest child 0 to 17',
    10:'Single-earner couple, female employed, youngest child 18 to 24',
    11:'Non-earner couple, no children or none under 25',
    12:'Non-earner couple, youngest child 0 to 17',
    13:'Non-earner couple, youngest child 18 to 24',
    14:'Lone-parent family, parent employed, youngest child 0 to 17',
    15:'Lone-parent family, parent employed, youngest child 18 to 24',
    16:'Lone-parent family, parent not employed, youngest child 0 to 17',
    17:'Lone-parent family, parent not employed, youngest child 18 to 24',
    18:'Other families'
})
# Mapping the economic family type

data2['COWMAIN']=data2['COWMAIN'].map({1:'Public sector employees',2:'Private sector employees'})
# Mapping the class of worker

data2['PROV']=data2['PROV'].map({10:'NL',11:'PE',12:'NS',13:'NB',24:'QC',35:'ON',46:'MB',47:'SK',48:'AB',59:'BC'})
# Mapping the provinces

data2['AGE_12']=data2['AGE_12'].map({
    1:'15-19'
    ,2:'20-24'
    ,3:'25-29'
    ,4:'30-34'
    ,5:'35-39'
    ,6:'40-44'
    ,7:'45-49'
    ,8:'50-54'
    ,9:'55-59'
    ,10:'60-64'
    ,11:'65-69'
    ,12:'70+'})
# Mapping the age groups

data2['EDUC']=data2['EDUC'].map({
    0:'0 to 8 years'
    ,1:'Some high school'
    ,2:'High school graduate'
    ,3:'Some post-secondary'
    ,4:'Post-secondary certificate or diploma'
    ,5:'Bachelor\'s degree'
    ,6:'Above bachelor\'s degree'})
# Mapping the education levels

data2['NAICS_21']=data2['NAICS_21'].map({
1:'Agriculture'
,2:'Forestry and logging and support activities for forestry'
,3:'Fishing, hunting and trapping'
,4:'Mining, quarrying, and oil and gas extraction'
,5:'Utilities'
,6:'Construction'
,7:'Manufacturing - durable goods'
,8:'Manufacturing - non-durable goods'
,9:'Wholesale trade'
,10:'Retail trade'
,11:'Transportation and warehousing'
,12:'Finance and insurance'
,13:'Real estate and rental and leasing'
,14:'Professional, scientific and technical services'
,15:'Business, building and other support services'
,16:'Educational services'
,17:'Health care and social assistance'
,18:'Information, culture and recreation'
,19:'Accommodation and food services'
,20:'Other services (except public administration)'
,21:'Public administration' 
})
# Mapping the industry

data2['FIRMSIZE']=data2['FIRMSIZE'].map({
     1:'Less than 20 employees'
    ,2:'20-99 employees'
    ,3:'100-500 employees'
    ,4:'More than 500 employees'})
# Mapping the firm size

data2['UNION']=data2['UNION'].map({
    1:'Union member',
    2:'Not a member but covered by a union contract or collective agreement',
    3:'Non-unionized'})
# Mapping the union status

data2['NOC_43']=data2['NOC_43'].map({
    1:'Legislative and senior management occupations',
    2:'Specialized middle management occupations',
    3:'Middle management occupations in retail and wholesale trade and customer services',
    4:'Middle management occupations in trades, transportation, production and utilities',
    5:'Professional occupations in finance',
    6:'Professional occupations in business',
    7:'Administrative and financial supervisors and specialized administrative occupations',
    8:'Administrative occupations and transportation logistics occupations',
    9:'Administrative and financial support and supply chain logistics occupations',
    10:'Professional occupations in natural sciences',
    11:'Professional occupations in applied sciences (except engineering)',
    12:'Professional occupations in engineering',
    13:'Technical occupations related to natural and applied sciences',
    14:'Health treating and consultation services professionals',
    15:'Therapy and assessment professionals',
    16:'Nursing and allied health professionals',
    17:'Technical occupations in health',
    18:'Assisting occupations in support of health services',
    19:'Professional occupations in law',
    20:'Professional occupations in education services',
    21:'Professional occupations in social and community services',
    22:'Professional occupations in government services',
    23:'Occupations in front-line public protection services',
    24:'Paraprofessional occupations in legal, social, community and education services',
    25:'Assisting occupations in education and in legal and public protection',
    26:'Care providers and public protection support occupations and student monitors, crossing guards and related occupations',
    27:'Professional occupations in art and culture',
    28:'Technical occupations in art, culture and sport',
    29:'Occupations in art, culture and sport',
    30:'Support occupations in art, culture and sport',
    31:'Retail sales and service supervisors and specialized occupations in sales and services',
    32:'Occupations in sales and services',
    33:'Sales and service representatives and other customer and personal services occupations',
    34:'Sales and service support occupations',
    35:'Technical trades and transportation officers and controllers',
    36:'General trades',
    37:'Mail and message distribution, other transport equipment operators and related maintenance workers',
    38:'Helpers and labourers and other transport drivers, operators and labourers',
    39:'Supervisors and occupations in natural resources, agriculture and related production',
    40:'Workers and labourers in natural resources, agriculture and related production',
    41:'Supervisors, central control and process operators in processing, manufacturing and utilities and aircraft assemblers and inspectors',
    42:'Machine operators, assemblers and inspectors in processing, manufacturing and printing',
    43:'Labourers in processing, manufacturing and utilities'
})
# Mapping the occupation

data2['NOC_10']=data2['NOC_10'].map({
    1:'Management occupations',
    2:'Business, finance and administration occupations, except management',
    3:'Natural and applied sciences and related occupations, except management',
    4:'Health occupations, except management',
    5:'Occupations in education, law and social, community and government services, except management',
    6:'Occupations in art, culture, recreation and sport, except management',
    7:'Sales and service occupations, except management',
    8:'Trades, transport and equipment operators and related occupations, except management',
    9:'Natural resources, agriculture and related production occupations, except management',
    10:'Occupations in manufacturing and utilities, except management'})
# Mapping the occupation


### <span style="color: #8FBC8F;">Saving Checkpoint 1</span>

In [19]:
data3=data2.copy()
# Creating a copy of the data

###  <span style="color: #4682B4;">Splitting the Data into Training and Testing</span>


In [20]:
labels=data3['HRLYEARN']
#Creating the label for splitting the data

features=data3.drop(columns=['HRLYEARN'],axis=1)
# Creating the features for splitting the data

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, train_size=.7, random_state=42)
# Splitting the features and labels into training and testing datasets
# The training dataset is 80% of the data and the testing dataset is 20% of the data
# This is to ensure no data leekage occurs, and to ensure the model is not overfitting
# X_train will be used as the training set and the validation set for machine learning later on

### <span style="color: #BDB76B;">Feature Selection: Correlation Analysis</span> 


In [21]:
#profile2 = ProfileReport(X_train, title="Profiling Report")
#profile2.to_file("EDA after removing non-correlated features.html")
# Creating a another profile report to examine the correlation just on the training data

In [22]:
# Will be dropping the remaining measures becauase of high correlation with other variables
# This is removed from the X_train variable

remove_attributes2=[]
# Creating an empty list to store the variables to be removed
remove_attributes2.append('NOC_43')
# Correlated with COWMAIN, NOCS_10, SEX
remove_attributes2.append('NOC_10')
# Correlated with SEX, NOC_43, NAICS_21
remove_attributes2.append('ESTSIZE')
# Dropping MJH because it only contains one value, so it is not useful for the model
remove_attributes2.append('AHRSMAIN')
# Correlated with UTOTHRS, UHRSMAIN, ATOTHRS
remove_attributes2.append('UTOTHRS')
# Correlated with  ATOTHRS, UHRSMAIN, AHRSMAIN
remove_attributes2.append('ATOTHRS')
# Correlated with UTOTHRS, UHRSMAIN, ATOTHRS
remove_attributes2.append('COWMAIN')
# Removing cowmain because it is correlated with NAICS_21, UNION and NOC_43
remove_attributes2.append('EFAMTYPE')
# Removing efamtype because it is correlated with MASRSTAT

In [23]:
X_train=X_train.drop(columns=remove_attributes2)
X_test=X_test.drop(columns=remove_attributes2)
# Dropping the variables with high correlation in both sets

In [24]:
#profile3 = ProfileReport(X_train, title="Profiling Report")
#profile3.to_file("EDA after removing the correlated features.html")
# Dropping the variables with a high correlatoion and making another profile report

In [25]:
master_list_of_variables_removed=remove_attributes+remove_attributes2
null_attributes_full_variable_name=[]
for attribute in master_list_of_variables_removed:
    null_attributes_full_variable_name.append(full_variable_name_mapping.get(attribute.lower()))
# Compiling all the variables that were removed from the dataset

null_attributes_table=pd.DataFrame({'Variable':master_list_of_variables_removed,'Description':null_attributes_full_variable_name})
# Creating a null attributes table

###  <span style="color: #4682B4;">Combining the X_train and y_train into one Dataframe and X_test and y_test</span>


In [26]:
data3=X_train.join(y_train, how='inner')

print(data3.info())
# Joing back the training labels to the training set to have a complete dataset and make it easier to work with.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108675 entries, 336295 to 347909
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   PROV      108675 non-null  object 
 1   CMA       108675 non-null  int64  
 2   AGE_12    108675 non-null  object 
 3   SEX       108675 non-null  int64  
 4   MARSTAT   108675 non-null  int64  
 5   EDUC      108675 non-null  object 
 6   IMMIG     108675 non-null  int64  
 7   NAICS_21  108675 non-null  object 
 8   UHRSMAIN  108675 non-null  float64
 9   TENURE    108675 non-null  float64
 10  UNION     108675 non-null  object 
 11  FIRMSIZE  108675 non-null  object 
 12  HRLYEARN  108675 non-null  float64
dtypes: float64(3), int64(4), object(6)
memory usage: 15.6+ MB
None


In [27]:
test_data=X_test.join(y_test,how='inner')
# Joing back the test labels to the test set to have a complete dataset and make it easier to work with.

In [28]:
#Creating a dataframe with the all the unique values in data3 and the number of classes in the categorical columns
unique_values_table=pd.DataFrame()
for column in data3.columns:
    unique_values_table[column]=[data3[column].nunique()]
unique_values_table=unique_values_table.T
unique_values_table=unique_values_table.rename(columns={0:'Unique Values'})
unique_values_table=unique_values_table.reset_index()
unique_values_table=unique_values_table.rename(columns={'index':'Variables'})
unique_values_table=unique_values_table.sort_values(by='Unique Values',ascending=False)
unique_values_table.reset_index(drop=True,inplace=True)
#unique_values_table.to_excel('Unique Values Table_Figure_4.xlsx',index=False)

### <span style="color: #8FBC8F;">Saving Checkpoint 2</span>

In [29]:
data4=data3.copy()
#Creating a checkpoint for the data

###  <span style="color: #4682B4;">Creating Dummy Variables and Scaling</span>
 

##### Renaming Age_12 and NAICS_21 variables

In [30]:
data4.rename(columns={'AGE_12': 'AGE'}, inplace=True)
# Renaming the age column
data4.rename(columns={'NAICS_21':'NAICS'},inplace=True)
# Renaming the age column


##### OneHotEncoder and MinMaxScaler

In [31]:
categorical_features = ['PROV', 'CMA', 'AGE', 'SEX', 'MARSTAT', 'EDUC', 'IMMIG', 'NAICS', 'UNION', 'FIRMSIZE']
continuous_features = ['UHRSMAIN', 'TENURE']

y = data4['HRLYEARN']
X = data4.drop(columns=['HRLYEARN'])

# Settting up the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Transforming X
X_transformed = preprocessor.fit_transform(X)

# Checking the shape of the X
#print("Shape of X:", X_transformed.shape)

# Get feature names after transformation
categorical_transformer = preprocessor.named_transformers_['cat']
onehot_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
feature_names = np.append(continuous_features, onehot_feature_names)

# Counting of feature names
#print("Number of feature names:", len(feature_names))

# Converting tge sparse matrix to a dense matrix
X_transformed_dense = X_transformed.toarray()

# Output the transformed features with the original index
features = pd.DataFrame(X_transformed_dense, columns=feature_names, index=data4.index)



In [32]:
#print(features.shape)
#print(y.shape)

###  <span style="color: #4682B4;">Random Forest Graph of Feature Importance</span>
  

In [33]:
'''
# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Parameter Grid
param_grid = {
    'model__n_estimators': [10],
    'model__max_features': [1.0],
    'model__max_depth': [None],
    'model__min_samples_split': [2],
    'model__min_samples_leaf': [1],
    'model__bootstrap': [True]
}
# KFolds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# RamdomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,n_iter=1,cv=kf,random_state=42, n_jobs=-1)

# Fit the model
random_search.fit(X, y)

my_model = random_search.best_estimator_

# Getting the model trained
model = my_model.named_steps['model']

# Retreiving the feature names
categorical_transformer = my_model.named_steps['preprocessor'].named_transformers_['cat']
onehot_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
feature_names = np.append(continuous_features, onehot_feature_names)

# Extracted the feature importances from the model
feature_importances = model.feature_importances_

# Created a DataFrame for better visualization
features_data_frame = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
features_data_frame = features_data_frame.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=features_data_frame, palette='viridis')
plt.title('Feature Importances from Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()
'''


"\n# Pipeline\npipeline = Pipeline(steps=[\n    ('preprocessor', preprocessor),\n    ('model', RandomForestRegressor(random_state=42))\n])\n\n# Parameter Grid\nparam_grid = {\n    'model__n_estimators': [10],\n    'model__max_features': [1.0],\n    'model__max_depth': [None],\n    'model__min_samples_split': [2],\n    'model__min_samples_leaf': [1],\n    'model__bootstrap': [True]\n}\n# KFolds\nkf = KFold(n_splits=10, shuffle=True, random_state=42)\n\n# RamdomizedSearchCV\nrandom_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,n_iter=1,cv=kf,random_state=42, n_jobs=-1)\n\n# Fit the model\nrandom_search.fit(X, y)\n\nmy_model = random_search.best_estimator_\n\n# Getting the model trained\nmodel = my_model.named_steps['model']\n\n# Retreiving the feature names\ncategorical_transformer = my_model.named_steps['preprocessor'].named_transformers_['cat']\nonehot_feature_names = categorical_transformer.get_feature_names_out(categorical_features)\nfeature_names = np.append(c

###  <span style="color: #4682B4;">Checking for Normality</span>
   

##### Creating Histograms of the Quantititaive Variables

In [34]:
def create_histogram(feature_series, titles, x_labels):
    # Create subplots
    fig, ax = plt.subplots(ncols=len(feature_series), nrows=1, figsize=(20, 7))

    # Plot each feature in a separate subplot
    for i, axi in enumerate(ax.flat):
        try:
            axi.hist(feature_series[i], bins=50, color='skyblue', edgecolor='black')
            axi.set_title(titles[i], fontsize=14)
            axi.set_xlabel(x_labels[i], fontsize=12)
            axi.set_ylabel('Frequency', fontsize=12)
        except IndexError:
            break
    plt.tight_layout()
    plt.show()


In [35]:
#create_histogram([data4['HRLYEARN'], data4['UHRSMAIN'], data4['TENURE']],['Hourly Earnings (in dollars)', 'Usual Hours Worked (per week)', 'Job Tenure (in months)'], ['Dollars', 'Hours', 'Months'])

##### Shaprio-Wilk Test

In [36]:
# Perform Shapiro-Wilk test to test for normality for UHRSMAIN and TENURE
for column in ['UHRSMAIN','TENURE']:
    stat, p = stats.shapiro(data4[column])
    #print(f'Statistic: {stat}, p-value: {p}')
    pass

    alpha = 0.05
    if p > alpha:
        #print(f'{column} looks like a normal distribution (fail to reject H0)')
        pass
    else:
        #print(f'{column} does not look like a normal distribution (reject H0)')
        pass
# Perform Shapiro-Wilk test to test for HRLYYEARN training data
stat, p = stats.shapiro(data4['HRLYEARN'])
if p > alpha:
    #print(f'HRLYEARN looks like a normal distribution (fail to reject H0)')
    pass
else:
    #print(f'HRLYEARN does not look like a normal distribution (reject H0)')
    pass

/Users/hk/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


###  <span style="color: #4682B4;">Linearity Assumptions Tested</span>
    

##### No Log Transformation of the Dependent Variable

In [37]:
y=data4['HRLYEARN']
#The response variable

# Fitting the model
model = sm.OLS(y, features).fit()

model_summary = model.summary()
#print(model_summary)

# Getting the residuals
residuals = model.resid
fitted_values=model.fittedvalues

# Checking for independence (Durbin-Watson test)
dw_test = sm.stats.stattools.durbin_watson(residuals)
#print('Durbin-Watson test statistic:', dw_test)

shapiro_test = stats.shapiro(residuals)
#print('Shapiro-Wilk test statistic:', shapiro_test)

mean_residuals = residuals.mean()
#print('Mean of the residuals:', mean_residuals)

/Users/hk/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


##### Log Transformation of the Dependent Variable

In [38]:
# Apply log transformation to the dependent variable to fix the normality issue and check the residuals again
# This fixed the normaility issue but not the homoscedasticity issue
 
#transformed dependent variable (y)
log_y=np.log(y)

# Fit the model
log_model = sm.OLS(log_y, features).fit()

# Get the residuals
log_residuals = model.resid

log_fitted_values = log_model.fittedvalues

# Check for independence (Durbin-Watson test)
dw_test = sm.stats.stattools.durbin_watson(residuals)
#print('Durbin-Watson test statistic:', dw_test)

shapiro_test = stats.shapiro(residuals)
#print('Shapiro-Wilk test statistic:', shapiro_test)

# Check for mean of residuals equal to zero
mean_residuals = residuals.mean()
#print('Mean of residuals:', mean_residuals)



##### Histograms of No Log, and Log Tranformations of the Dependent Variable

In [39]:
#create_histogram([residuals,log_residuals],['Histogram of Residuals', 'Histogram of Log Residuals'],['Residuals','Residuals'])

##### Scatter Plots to Visualize the Common Variance of the Residuals

In [40]:
def create_scatterplot(x,y, titles):
    # Create subplots
    fig, ax = plt.subplots(ncols=len(x), nrows=1, figsize=(20, 7))

    # Plot each feature in a separate subplot
    for i, axi in enumerate(ax.flat):
        try:
            axi.scatter(x=x[i],y=y[i], color='skyblue', edgecolor='black')
            axi.set_title(titles[i], fontsize=14)
            axi.set_xlabel("Fitted Values", fontsize=12)
            axi.set_ylabel('Residulas', fontsize=12)
        except IndexError:
            break
    plt.tight_layout()
    plt.show()

#create_scatterplot([fitted_values,log_fitted_values], [residuals,log_residuals],['Fitted values vs Residuals','Log Fitted values vs Residuals'])


###  <span style="color: #4682B4;">Log Transform the Dependent Value</span>
     

In [41]:
data4['HRLYEARN']=np.log(data3['HRLYEARN'])

###  <span style="color: #4682B4;">Outlier Detection using IQR</span>
      

##### Creating Boxplots

In [42]:
# Feature series to be plotted
def create_box_plot(feature_series,feature_labels):
    # Customize flier properties for outliers
    flierprops = dict(marker='o', markerfacecolor='r', markersize=12,
                    linestyle='none', markeredgecolor='g')
    # Create subplots
    fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 7))

    # Plot each feature in a separate subplot
    for i, axi in enumerate(ax.flat):
        try:
            axi.boxplot(feature_series[i], flierprops=flierprops, patch_artist=True,
                        boxprops=dict(facecolor='skyblue', color='black'),
                        capprops=dict(color='black'),
                        whiskerprops=dict(color='black'),
                        medianprops=dict(color='red'))
            axi.set_title(feature_labels[i], fontsize=14)
            axi.tick_params(axis='y', labelsize=12)
            axi.tick_params(axis='x', labelsize=0)
        except IndexError:
            break
    plt.tight_layout()
    plt.show()

In [43]:
# Displaying the box plots
#create_box_plot([data4['HRLYEARN'], data4['UHRSMAIN'], data4['TENURE']],['Hourly Earnings (in dollars)', 'Usual Hours Worked (per week)', 'Job Tenure (in months)'])

##### Creating The 5 Number Summaries for the Quantitative Variables

In [44]:
# 5 number summary for HRLYEARN.
Q1_HRLYEARN = np.percentile(data3['HRLYEARN'], 25)
Q2_HRLYEARN  = np.percentile(data3['HRLYEARN'], 50)
Q3_HRLYEARN  = np.percentile(data3['HRLYEARN'], 75)
min_value_HRLYEARN  = np.min(data3['HRLYEARN'])
max_value_HRLYEARN  = np.max(data3['HRLYEARN'])
IQR_HRLYEARN = Q3_HRLYEARN  - Q1_HRLYEARN #print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_HRLYEARN ,Q2_HRLYEARN ,Q3_HRLYEARN , min_value_HRLYEARN ,max_value_HRLYEARN , IQR_HRLYEARN)

In [45]:
Q1_UHRSMAIN = np.percentile(data3['UHRSMAIN'], 25)
Q2_UHRSMAIN = np.percentile(data3['UHRSMAIN'], 50)
Q3_UHRSMAIN = np.percentile(data3['UHRSMAIN'], 75)
min_value_UHRSMAIN = np.min(data3['UHRSMAIN'])
max_value_UHRSMAIN = np.max(data3['UHRSMAIN'])
IQR_UHRSMAIN = Q3_UHRSMAIN - Q1_UHRSMAIN
#print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_UHRSMAIN,Q2_UHRSMAIN,Q3_UHRSMAIN, min_value_UHRSMAIN,max_value_UHRSMAIN, IQR_UHRSMAIN)


In [46]:
Q1_TENURE = np.percentile(data3['TENURE'], 25)
Q2_TENURE = np.percentile(data3['TENURE'], 50)
Q3_TENURE = np.percentile(data3['TENURE'], 75)
min_value_TENURE = np.min(data3['TENURE'])
max_value_TENURE = np.max(data3['TENURE'])
IQR_TENURE = Q3_TENURE - Q1_TENURE
#print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_TENURE,Q2_TENURE,Q3_TENURE, min_value_TENURE,max_value_TENURE, IQR_TENURE)


##### Number of Outliers

In [47]:
# Checking for outliers in the response variable using IQR
outliers_HRLYEARN = data3[(data3['HRLYEARN'] < (Q1_HRLYEARN - 1.5 * IQR_HRLYEARN)) | (data3['HRLYEARN'] > (Q3_HRLYEARN + 1.5 * IQR_HRLYEARN))]
outliers_UHRSMAIN = data3[(data3['UHRSMAIN'] < (Q1_UHRSMAIN - 1.5 * IQR_UHRSMAIN)) | (data3['UHRSMAIN'] > (Q3_UHRSMAIN + 1.5 * IQR_UHRSMAIN))]
outliers_TENURE = data3[(data3['TENURE'] < (Q1_TENURE - 1.5 * IQR_TENURE)) | (data3['TENURE'] > (Q3_TENURE + 1.5 * IQR_TENURE))]
#print("The number of outliers for HRLYEARN, UHRSMAIN, and TENURE are:",len(outliers_HRLYEARN),len(outliers_UHRSMAIN),len(outliers_TENURE))
#print("The shape of the working dataset is:",data3.shape)
#The outliers account for approximately 18% of the training data. The outliers will be removed from the dataset.

### <span style="color: #8FBC8F;">Saving Checkpoint 3</span> 

In [48]:
data5=data4.copy()
#data5.info()
# Creating new data frame after removing the outliers
# Crating a newdataframe to remove the outliers from data3

### <span style="color: #BDB76B;"> Removing Outliers from the Quantitative Variables</span> 


In [49]:
#removing the outliers from the dataset
data5 = data5[(data5['HRLYEARN'] >= (Q1_HRLYEARN - 1 * IQR_HRLYEARN)) & (data5['HRLYEARN'] <= (Q3_HRLYEARN + 1.5 * IQR_HRLYEARN))]
data5 = data5[(data5['UHRSMAIN'] >= (Q1_UHRSMAIN - 1 * IQR_UHRSMAIN)) & (data5['UHRSMAIN'] <= (Q3_UHRSMAIN + 1.5 * IQR_UHRSMAIN))]
data5 = data5[(data5['TENURE'] >= (Q1_TENURE - 1 * IQR_TENURE)) & (data5['TENURE'] <= (Q3_TENURE + 1.5 * IQR_TENURE))]

In [50]:
summary_statistics_table_continuous_variables_before_outliers=statistics_table(data4[['HRLYEARN','UHRSMAIN','TENURE']])
summary_statistics_table_continuous_variables_after_outliers=statistics_table(data5[['HRLYEARN','UHRSMAIN','TENURE']])
# Creating the summary statistics table for the continuous variables before and after removing the outliers
#print(summary_statistics_table_continuous_variables_before_outliers)
#print(summary_statistics_table_continuous_variables_after_outliers)

#summary_statistics_table_continuous_variables_before_outliers.to_excel('Continuous Variables Before Removing outliers.xlsx',index=False)  
#summary_statistics_table_continuous_variables_after_outliers.to_excel('Continuous Variables After Removing outliers.xlsx',index=False)  
# Exporting the summary statistics table for the continuous variables before and after removing the outliers

## <span style="color: #2F4F4F;">Apriori Algorithm</span>


### <span style="color: #8FBC8F;">Copying the data from Checkpoint 3</span>  

In [51]:
categorical_data= data5.copy()

### <span style="color: #4682B4;">  Apriori Pre-Processing</span>
 

#### <span>Creating Bins for the Aprirori Algorithm</span>

In [52]:
def bin_creator(numerical_variables):

    if isinstance(numerical_variables, pd.Series):
        variables=numerical_variables.to_frame()
        bin_maker = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
        variables=bin_maker.fit_transform(variables)
        # Getting the bin intervals
        bin_edges = bin_maker.bin_edges_
        # Creating a dictionary to map feature names to the respective bin intervals
        bin_edges_dict = {feature: edges for feature, edges in zip(numerical_variables, bin_edges)}
        # Printing bin interval for each feature
        for feature, edges in bin_edges_dict.items():
            #print(f"Bin interval for {feature}: {edges}")
            pass
        bin_info = []
        for feature, edges in bin_edges_dict.items():
            for i in range(len(edges) - 1):
                bin_info.append({
                    'Feature': feature,
                    'Bin Index': i,
                    'Bin Start': edges[i],
                    'Bin End': edges[i + 1]
                })
        # Creating a DataFrame 
        bin_info_df = pd.DataFrame(bin_info)
        # Displaying the DataFrame
        print(bin_info_df)
        #bin_info_df.to_excel('Binning Information Table.xlsx',index=False)
        return variables

    else:
        columns=list(numerical_variables.columns)
        variables=numerical_variables[columns]
        # Initializing the KBinsDiscretizer with 'quantile' strategy so that the labels are balanced. 
        bin_maker = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
        # Fitting and transforming 
        variables[columns]= bin_maker.fit_transform(variables)
        # Getting the bin intervals
        bin_edges = bin_maker.bin_edges_
        # Creating a dictionary to map feature names to the respective bin intervals
        bin_edges_dict = {feature: edges for feature, edges in zip(numerical_variables, bin_edges)}
        # Printing bin interval for each feature
        for feature, edges in bin_edges_dict.items():
            #print(f"Bin interval for {feature}: {edges}")
            pass
        bin_info = []
        for feature, edges in bin_edges_dict.items():
            for i in range(len(edges) - 1):
                bin_info.append({
                    'Feature': feature,
                    'Bin Index': i,
                    'Bin Start': edges[i],
                    'Bin End': edges[i + 1]
                })
        # Creating a DataFrame 
        bin_info_df = pd.DataFrame(bin_info)
        # Display the DataFrame
        #print(bin_info_df)
        #bin_info_df.to_excel('Binning Information Table.xlsx',index=False)
        return variables[columns]

training_bins=bin_creator(categorical_data[['UHRSMAIN', 'TENURE','HRLYEARN']])

/Users/hk/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:313: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


#### <span >  Creating the Dummy Variables for the Apriori Alogirthm</span>
  

In [53]:
def dummy_creator(dummy, dummy2):
    columns = list(dummy.columns)
    dummy_creator = OneHotEncoder(sparse=False)
    encoded_features = dummy_creator.fit_transform(dummy)
    feature_names = dummy_creator.get_feature_names_out(columns)
    categorical_data = pd.DataFrame(encoded_features, columns=feature_names, index=dummy.index)
    
    remaining_columns = dummy2.drop(columns=columns)
    other_columns = list(remaining_columns.columns)
    encoded_other_features = dummy_creator.fit_transform(remaining_columns)
    other_feature_names = dummy_creator.get_feature_names_out(other_columns)
    other_categorical_data = pd.DataFrame(encoded_other_features, columns=other_feature_names, index=remaining_columns.index)
    
    combined_data = categorical_data.join(other_categorical_data, how='inner')
    return combined_data

categorical_data = dummy_creator(training_bins, categorical_data)

/Users/hk/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/hk/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### <span style="color: #4682B4;"> Apriori Replication</span>
  

In [54]:
# Applying the Apriori algorithm
frq_itemset = apriori(categorical_data, min_support=0.05, use_colnames=True)
#print("Frequent Itemsets:\n", frq_itemset.head())
# Support set ato 5% because 25% was too high

# Association Rules
rules = association_rules(frq_itemset, metric="confidence", min_threshold=0.01)
#print("Association rules:\n", rules.head())
#Confidence set 1% because 80% was too high

# Defining Right Hand Side
rhs = ['HRLYEARN_3.0', 'HRLYEARN_4.0']

# Filtering the rules by RHS
rules = rules[rules['consequents'].apply(lambda x: any(item in x for item in rhs))]

print("Association Rules:\n", rules)

/Users/hk/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Association Rules:
                                              antecedents  \
26                                        (UHRSMAIN_1.0)   
75                                        (UHRSMAIN_2.0)   
76                                        (UHRSMAIN_2.0)   
228                                         (TENURE_3.0)   
254                                         (TENURE_4.0)   
357                                            (PROV_ON)   
359                                              (CMA_0)   
361                                              (CMA_1)   
363                                              (SEX_0)   
365                                              (SEX_1)   
367                                          (MARSTAT_0)   
369                                          (MARSTAT_1)   
371                             (EDUC_Bachelor's degree)   
373         (EDUC_Post-secondary certificate or diploma)   
375                                            (IMMIG_0)   
377                 

In [55]:
# Generating a report with the important rules
report = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

#report.to_excel('rules_hrlyearn.xlsx', index=False)

### <span style="color: #4682B4;">  Change</span>
   

## <span style="color: #2F4F4F;">Non-Linear Regression</span>

### <span style="color: #4682B4;">  Creating the Features and Lables For Non-Linear Regeression</span>


In [56]:
features=data5.copy()
features=features.drop(columns=['HRLYEARN'])

#dropping these features because they are scaled, and we need to keep the original values for the non-linear models, and scaled later.
continuous_features=data5[['TENURE', 'UHRSMAIN']]

label = data5['HRLYEARN']
#These will be used for the non-linear machine learning algoirthms. 

### <span style="color: #4682B4;">Non-Linear Regression Replication</span>
 

##### Non-Linear Train, Test, Split for Base Models without Hyper Tuning

In [57]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=42)

#### <span>  Non-Linear Regression Pre-processing for Base Models without Hyper Tuning</span>
 

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), list(continuous_features.columns)),
        ('cat', OneHotEncoder(drop='first'), list(features.drop(columns=['TENURE', 'UHRSMAIN']).columns))
    ])

#### Non-Linear Base Models without Hyper Tuning

In [85]:
svr_model=SVR(kernel='rbf')
rf_model=RandomForestRegressor(n_estimators=100,random_state=42)

xg_model=XGBRegressor(seed=42)
rt_model=DecisionTreeRegressor(random_state=42)
knn_model=KNeighborsRegressor()

models=[rf_model,svr_model,xg_model,knn_model,rt_model]
#models=[knn_model]

train_score=[]
test_score=[]
model_times = []

for model in models:
    pipeline=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    end_time = time.time()
    predictions = pipeline.predict(X_test)
    train_score.append(mean_squared_error(y_test, predictions, squared=False))

    model_times.append(end_time - start_time)

    print(f"RMSE for {model}:", mean_squared_error(y_test, predictions, squared=False))
    print(f"Time taken for {model}: {end_time - start_time} seconds")
    
print(pd.DataFrame({'Train Score':list(np.array(train_score).reshape(-1,1))}, index=models))

RMSE for RandomForestRegressor(random_state=42): 0.26215338362758844
Time taken for RandomForestRegressor(random_state=42): 427.15135526657104 seconds


 #### <span>Kruskal-Wallis Test for Non-Linear Regression</span>

In [65]:
data_nlr_results = pd.DataFrame({ 
    'Model': ['Random Forest', 'Support Vector', 'XGBoost', 'KNN', 'Regression Tree'],
    'RMSE': [0.26, 0.31, 0.31, 0.33, 0.35]
})

# Perform the Kruskal-Wallis test
statistic, p_value = kruskal(
    data_nlr_results[data_nlr_results['Model'] == 'Random Forest']['RMSE'],
    data_nlr_results[data_nlr_results['Model'] == 'Support Vector']['RMSE'],
    data_nlr_results[data_nlr_results['Model'] == 'XGBoost']['RMSE'],
    data_nlr_results[data_nlr_results['Model'] == 'KNN']['RMSE'],
    data_nlr_results[data_nlr_results['Model'] == 'Regression Tree']['RMSE']
)

#print(f"Kruskal-Wallis test statistic: {statistic}")
#print(f"P-value: {p_value}")

ShapiroResult(statistic=0.9327995777130127, pvalue=0.615582287311554)


#### One-Way Anova

In [81]:
RMSE=[0.26, 0.31, 0.31, 0.33, 0.35]
stats.shapiro(RMSE)
# Fit the model
model = ols('RMSE ~ C(Model)', data=data_nlr_results).fit()

# Anova
anova_results = anova_lm(model)
print(anova_results)


           df        sum_sq  mean_sq    F  PR(>F)
C(Model)  4.0  4.480000e-03  0.00112  0.0     NaN
Residual  0.0  1.201780e-31      inf  NaN     NaN


/Users/hk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/anova.py:138: RuntimeWarning: divide by zero encountered in scalar divide
  (model.ssr / model.df_resid))


### <span style="color: #4682B4;"> Non-Linear Regression Hyper Tuned</span>
  

##### Non-Linear Regression Hyper Tuned

In [ ]:
#This is Non-Linear Regression hyper tuning
continuous_features = ['UHRSMAIN', 'TENURE']
categorical_features = ['PROV', 'CMA', 'AGE', 'SEX', 'MARSTAT', 'EDUC', 'IMMIG', 'NAICS', 'UNION', 'FIRMSIZE']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])
#Setting up the column transformer

kf = KFold(n_splits=5, shuffle=True, random_state=42)
#Initializing K-Folds

param_grids = {
    'XGB': {
        'model__n_estimators': np.arange(100, 1100, 1),
        'model__learning_rate': np.logspace(-3, 0, 100),
        'model__max_depth': np.arange(1, 11, 1),
        'model__subsample': np.arange(0.5, 1.0, 0.1),
        'model__colsample_bytree': np.arange(0.5, 1.0, 0.1),
        'model__gamma': np.arange(0, 5, 1),
        'model__reg_alpha': np.logspace(-3, 2, 100),
    },
    'DT': {
        'model__criterion': ['mse', 'mae'],
        'model__splitter': ['best', 'random'],
        'model__max_depth': np.arange(1, 11, 1),
        'model__min_samples_split': np.arange(2, 10, 1),
        'model__min_samples_leaf': np.arange(1, 10, 1)
    },
    'RF': {
        'model__n_estimators': np.arange(100, 1100, 1),
        'model__max_features': ['sqrt', 'log2'],
        'model__max_depth': np.arange(1, 110, 1),
        'model__min_samples_split': np.arange(2, 10, 1),
        'model__min_samples_leaf': np.arange(1, 10, 1)

    },
    'SVR': {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__gamma': ['scale', 'auto'],
        'model__C': np.logspace(-3,2,100)
    },
    'KNN': {
        'model__n_neighbors': np.arange(1, 20, 1),
        'model__weights': ['uniform', 'distance'],
        'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'model__p': np.arange(1, 5, 1)

    }
}
#Defining the parameter grids for each model


models = {
    'XGB': XGBRegressor(seed=42)
    ,'DT': DecisionTreeRegressor(random_state=42)
    ,'RF': RandomForestRegressor(random_state=42)
    ,'SVR': SVR(random_state=42)
    ,'KNN': KNeighborsRegressor()
}
# Defining each model

# Perform RandomizedSearchCV for each model
scores = []
#creating an empty list to store the scores
times = []

for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    
    grid_search = RandomizedSearchCV(pipeline, param_distributions=param_grids[model_name], n_iter=5, cv=kf, random_state=42, scoring='neg_root_mean_squared_error',n_jobs=-1)
    start_time = time.time()
    grid_result = grid_search.fit(features, label)
    end_time = time.time()

    elapsed_time = end_time - start_time

    scores.append({
        'model': model_name,
        'best_score': grid_result.best_score_,
        'best_params': grid_result.best_params_
    })
#Running the randomized search for each model

    times.append({
        'model': model_name,
        'time': elapsed_time
    })

scores_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
# Creating a datafreame for all the scores
times_df = pd.DataFrame(times, columns=['model', 'time'])
print(scores_df)
print(times_df)

## <span style="color: #2F4F4F;">Classification</span>

### <span style="color: #4682B4;"> Buliding the Dataset for Classifiers</span>


In [82]:
# Creating the dependent variable into 5 categorical values
#features was created for non-linear regression
#continuous_features list that contains usual hours worked and tenure was created for non-linear regression

#print(label.head())
classified_labels=bin_creator(label)
#print("Label Shape: ", classified_labels.shape)
#print("Features Shape: ",features.shape
classified_labels.reshape(-1,1)

    Feature  Bin Index  Bin Start   Bin End
0  3.442019          0   1.934416  3.135494
1  3.442019          1   3.135494  3.382694
2  3.442019          2   3.382694  3.642836
3  3.442019          3   3.642836  3.912023
4  3.442019          4   3.912023  5.326322


array([[2.],
       [0.],
       [0.],
       ...,
       [2.],
       [0.],
       [3.]])

### <span style="color: #4682B4;"> Classifcation Replication</span>


#### <span>  Classification Train, Test, Split for Base Models without Hyper Tuning</span>


In [ ]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, classified_labels, test_size=0.3, random_state=42)

In [ ]:
#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

#### <span>  Classification Pre-processing for Base Models without Hyper Tuning</span>
 

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), list(continuous_features.columns)),
        ('cat', OneHotEncoder(drop='first'), list(features.drop(columns=['TENURE', 'UHRSMAIN']).columns))
    ])

#### <span>Classification Base Models without Hyper Tuning</span>
 

In [ ]:
svc_model=SVC(kernel='rbf',random_state=42)
rf_model=RandomForestClassifier(n_estimators=100,random_state=42)
nb_model = MultinomialNB()
lr_model=LogisticRegression(random_state=42)
xg_model=XGBClassifier(seed=42)
rt_model=DecisionTreeClassifier(random_state=42)

models=[rf_model,svc_model,nb_model,xg_model,lr_model,rt_model]

train_score=[]
test_score=[]
model_times = []
for model in models:
    pipeline=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    end_time = time.time()
    predictions = pipeline.predict(X_test)
    train_score.append(accuracy_score(y_test, predictions))
    model_times.append(end_time - start_time)

    print(f"Time taken for {model}: {end_time - start_time} seconds")
    print(f"Accuracy Score for {model}:", accuracy_score(y_test, predictions))
    print(f"Percision Score for {model}:", precision_score(y_test, predictions,average='weighted'))
    print(f"Recall Score for {model}:", recall_score(y_test, predictions,average='weighted'))
    print(f"F1-Score for {model}:", f1_score(y_test, predictions,average='weighted'))
    report = classification_report(y_test, predictions, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    #print(display(report_df))

    #print("\n")
    
print(pd.DataFrame({'Train Score':list(np.array(train_score).reshape(-1,1))}, index=models))

 #### <span>Friedman Test for Classification</span>

In [76]:
data_classification_results = pd.DataFrame({ 
    'Model': [
        'Random Forest', 
        'Decision Tree', 
        'XGBoost', 
        'SVC', 
        'Logistic Regression', 
        'MultinomialNB'
    ],
    'Accuracy': [70.3, 63.2, 50.2, 49.0, 42.6, 39.9],
    'Recall': [70.3, 63.2, 50.2, 49.0, 42.6, 39.9],
    'Precision': [70.2, 63.2, 49.4, 48.3, 41.3, 38.1],
    'F1-Score': [70.2, 63.2, 49.6, 48.5, 41.6, 38.4]
})

accuracy = data_classification_results['Accuracy'].values
recall = data_classification_results['Recall'].values
precision = data_classification_results['Precision'].values
f1_score = data_classification_results['F1-Score'].values


In [73]:
#Friedman Test
statistic, p_value = friedmanchisquare(accuracy, recall, precision, f1_score)

#print(f"Friedman T-Statistic: {statistic}")
#print(f"P-value: {p_value}")

 #### <span>Randomized Block Desgin</span>
 

In [75]:
for metric in [accuracy, recall, precision, f1_score]:
    #print(stats.shapiro(metric))
    pass
# The metrics are normally distributed.

# Levene's test for common variance 
levene_test = stats.levene(
    data_classification_results['Accuracy'],
    data_classification_results['Recall'],
    data_classification_results['Precision'],
    data_classification_results['F1-Score']
)
#print(f"Levene's test: Statistic={levene_test.statistic}, p-value={levene_test.pvalue}")
data_long = pd.melt(data_classification_results, id_vars=['Model'], var_name='Metric', value_name='Score')
model = ols('Score ~ C(Model) + C(Metric)', data=data_long).fit()
#Randomzied Block design
anova_= anova_lm(model)
print(anova_)

             df       sum_sq     mean_sq            F        PR(>F)
C(Model)    5.0  2961.478333  592.295667  4416.454847  3.362688e-23
C(Metric)   3.0     3.023333    1.007778     7.514499  2.678228e-03
Residual   15.0     2.011667    0.134111          NaN           NaN


### <span style="color: #4682B4;">  Classification Hyper Tuned</span>
  

#### <span >  Classification Models Hyper Tuned</span>
   

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), list(continuous_features.columns)),
        ('cat', OneHotEncoder(drop='first'), list(features.drop(columns=['TENURE', 'UHRSMAIN']).columns))
    ])
#Setting up the column transformer

kf = KFold(n_splits=5, shuffle=True, random_state=42)
#Initializing K-Folds

param_grids = {
    'svc': {
        'model__C': np.arange(1, 10, 1),
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__gamma': ['scale', 'auto']
        
    },
    'decision_tree': {
        'model__max_depth':np.arange(1, 10, 1),
        'model__min_samples_split':np.arange(2, 10, 1),
        'model__min_samples_leaf':np.arange(1, 10, 1)
    },
    'random_forest': {
        'model__n_estimators': np.arange(100, 1100, 1),
        'model__max_features': ['sqrt', 'log2'],
        'model__max_depth': np.arange(1, 110, 1),
        'model__min_samples_split': np.arange(2, 10, 1),
        'model__min_samples_leaf': np.arange(1, 10, 1)
        
    },
    'logistic_regression': {
            'model__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'model__C':np.logspace(-3,2,100),
            'model__penalty' : ['l1', 'l2', 'elasticnet', 'none']
    }
    ,
    'naive_bayes': {
        'model__alpha': [0.1, 0.5, 1, 2, 5]
    },
    'xgboost': {
        'model__n_estimators': [1,5,10,50,100,200,400,600,1200],
        'model__learning_rate': [0.1, 0.01, 0.001],
        'model__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'model__subsample': [0.7, 0.8, 0.9],
        'model__colsample_bytree': [0.7, 0.8, 0.9],
        'model__gamma': [0, 1, 5],
        'model__reg_alpha': [0, 1, 5]
    }
}
#Defining the parameter grids for each model


models = {
    'svc': SVC(random_state=42)
    ,'decision_tree': DecisionTreeRegressor(random_state=42)
    ,'naive_bayes': MultinomialNB()
    ,'xgboost': XGBClassifier(seed=42)
    ,'random_forest': RandomForestRegressor(random_state=42)
    ,'logistic_regression': LogisticRegression(random_state=42)

}
# Defining each model

# Perform RandomizedSearchCV for each model
scores = []
#creating an empty list to store the scores
times = []

for model_name, model in models.items():
    pipeline_classifier = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    grid_search = RandomizedSearchCV(pipeline_classifier, param_distributions=param_grids[model_name], n_iter=5, cv=kf, random_state=42, n_jobs=-1)
    start_time = time.time()
    grid_result = grid_search.fit(features, classified_labels)
    end_time = time.time()

    elapsed_time = end_time - start_time

    scores.append({
        'model': model_name,
        'best_score': grid_result.best_score_,
        'best_params': grid_result.best_params_
    })

    times.append({
        'model': model_name,
        'time': elapsed_time
    })

#Running the randomized search for each model

scores_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
# Creating a datafreame for all the scores
times_df = pd.DataFrame(times, columns=['model', 'time'])
print(scores_df)
print(times_df)
